In [1]:
# If this task has dependencies, list them them here
# (e.g. upstream = ['some_task']), otherwise leave as None.
upstream = None

# This is a placeholder, leave it as None
product = None

In [2]:
# Parameters
api_key = "ebc6d771a26e9b8009c65cb0ab76ba3d"
obs_start = "2015-01-01"
obs_stop = "2021-12-30"
units = "lin"
series_list = [
    "CROASMIDTIER",
    "VIXCLS",
    "TOTALSL",
    "CPIAUCSL",
    "M1REAL",
    "M1V",
    "WM1NS",
    "WM2NS",
    "DTWEXBGS",
    "SP500",
    "UMCSENT",
    "USSLIND",
]
fred_series_info_url = "https://api.stlouisfed.org/fred/series"
fred_series_observations_url = "https://api.stlouisfed.org/fred/series/observations"
product = {
    "nb": "/home/vgaurav/market_watch/output/notebooks/fetch_fred.ipynb",
    "data": "/home/vgaurav/market_watch/output/data/raw/fred_series_data.csv",
}


In [3]:
# your code here...

import requests
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
import json
from pathlib import Path

pd.options.display.max_rows=500
pd.options.display.max_columns=100
warnings.filterwarnings("ignore")

In [4]:
def fetch_fred_data(fred_url, series_list, api_key, obs_start, obs_stop, record_path, meta_columns=None):
    
    session = requests.Session()
    query_params =  {
        'file_type': 'json',
        'api_key' : api_key,
        'observation_start': obs_start,
        'observation_end' : obs_stop,
    }
    
    df_lst = []
    for series_id in tqdm(series_list):
        query_params['series_id'] = series_id
        response = session.get(
           fred_url,
           params=query_params
        )
        data = json.loads(response.text)
        df = pd.json_normalize(
            data, 
            record_path=record_path, 
            meta=meta_columns,
        )
        df['id'] = series_id
        df_lst.append(df)

    df = pd.concat(df_lst)
    
    return df

In [5]:
def get_series_info(fred_series_info_url, series_list, api_key, obs_start, obs_stop):
    
    record_path = ['seriess']
    df = fetch_fred_data(fred_series_info_url, series_list, api_key, obs_start, obs_stop, record_path)
    
    return df

In [6]:
def get_series_observations(fred_series_observations_url, series_list, api_key, obs_start, obs_stop):
    
    record_path = ['observations']
    df = fetch_fred_data(fred_series_observations_url, series_list, api_key, obs_start, obs_stop, record_path)
    df = df[['id','date','value']]
    return df

In [7]:
def clean_series_data(series_df):
    
    series_df.date = pd.to_datetime(series_df.date)
    # holidays are represented as . in dataset, let's replace them by null values
    series_df.value = series_df.value.replace('.', np.nan)
    # convert value to float as it comes as object
    series_df.value = series_df.value.astype(float)
    
    return series_df

In [8]:
def resample(series_df, method='ffill'):
    series_df = series_df.set_index('date')
    sampled = series_df.resample('B')
    sampled_df = sampled.interpolate(method=method)
    sampled_df = sampled_df.reset_index()
    return sampled_df

In [9]:
def convert_to_wide_format(series_obs_df):
    df_lst = []
    #  we need to resample each series individually becuase of date collission
    series_list = series_obs_df.id.unique()
    for series_id in series_list:
        # filter for given series id    
        filter_cond = series_obs_df['id'] == series_id
        # get individual series dataframe    
        series_df = series_obs_df[filter_cond]
        series_df = resample(series_df)
        series_df = series_df.pivot(index='date', columns='id', values='value')
        df_lst.append(series_df)
    
    df = pd.concat(df_lst, axis=1).sort_index()
    
    return df

In [10]:
series_info_df = get_series_info(fred_series_info_url, series_list, api_key, obs_start, obs_stop)
series_info_df.columns

  0%|                                                            | 0/12 [00:00<?, ?it/s]

  8%|████▎                                               | 1/12 [00:00<00:02,  5.06it/s]

 17%|████████▋                                           | 2/12 [00:01<00:10,  1.08s/it]

 33%|█████████████████▎                                  | 4/12 [00:02<00:03,  2.08it/s]

 42%|█████████████████████▋                              | 5/12 [00:02<00:02,  2.68it/s]

...

 83%|██████████████████████████████████████████▌        | 10/12 [00:03<00:00,  4.60it/s]

 92%|██████████████████████████████████████████████▊    | 11/12 [00:04<00:00,  1.85it/s]

100%|███████████████████████████████████████████████████| 12/12 [00:04<00:00,  2.27it/s]

100%|███████████████████████████████████████████████████| 12/12 [00:04<00:00,  2.58it/s]

Index(['id', 'realtime_start', 'realtime_end', 'title', 'observation_start',
       'observation_end', 'frequency', 'frequency_short', 'units',
       'units_short', 'seasonal_adjustment', 'seasonal_adjustment_short',
       'last_updated', 'popularity', 'notes'],
      dtype='object')

In [11]:
# need to call below functions in order
series_obs_df = get_series_observations(fred_series_observations_url, series_list, api_key, obs_start, obs_stop)
series_obs_df.columns

  0%|                                                            | 0/12 [00:00<?, ?it/s]

  8%|████▎                                               | 1/12 [00:02<00:25,  2.34s/it]

 17%|████████▋                                           | 2/12 [00:02<00:10,  1.09s/it]

 25%|█████████████                                       | 3/12 [00:03<00:08,  1.09it/s]

 33%|█████████████████▎                                  | 4/12 [00:04<00:06,  1.17it/s]

...

 83%|██████████████████████████████████████████▌        | 10/12 [00:10<00:02,  1.39s/it]

 92%|██████████████████████████████████████████████▊    | 11/12 [00:10<00:01,  1.01s/it]

100%|███████████████████████████████████████████████████| 12/12 [00:12<00:00,  1.16s/it]

100%|███████████████████████████████████████████████████| 12/12 [00:12<00:00,  1.02s/it]

Index(['id', 'date', 'value'], dtype='object')

In [12]:
series_obs_df = clean_series_data(series_obs_df)
series_obs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6713 entries, 0 to 61
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   id      6713 non-null   object        
 1   date    6713 non-null   datetime64[ns]
 2   value   6500 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 209.8+ KB


In [13]:
series_wide_df = convert_to_wide_format(series_obs_df)
series_wide_df = series_wide_df.fillna(method='ffill')
series_wide_df.columns

Index(['CROASMIDTIER', 'VIXCLS', 'TOTALSL', 'CPIAUCSL', 'M1REAL', 'M1V',
       'WM1NS', 'WM2NS', 'DTWEXBGS', 'SP500', 'UMCSENT', 'USSLIND'],
      dtype='object', name='id')

In [14]:
series_wide_df.tail(25)

id,CROASMIDTIER,VIXCLS,TOTALSL,CPIAUCSL,M1REAL,M1V,WM1NS,WM2NS,DTWEXBGS,SP500,UMCSENT,USSLIND
date,,,,,,,,,,,,
2021-11-26,324.308697,28.62,4411.21508,278.524,7282.5,1.185,20297.2,21416.5,116.5112,4594.62,67.4,1.57
2021-11-29,324.308697,22.96,4411.21508,278.524,7282.5,1.185,20324.5,21445.3,116.5858,4655.27,67.4,1.57
2021-11-30,324.308697,27.19,4411.21508,278.524,7282.5,1.185,20324.5,21445.3,116.2178,4567.00,67.4,1.57
2021-12-01,333.873647,31.12,4431.89944,280.126,7296.5,1.185,20324.5,21445.3,115.8992,4513.04,70.6,1.57
2021-12-02,333.873647,27.95,4431.89944,280.126,7296.5,1.185,20324.5,21445.3,116.0314,4577.10,70.6,1.57
2021-12-03,333.873647,30.67,4431.89944,280.126,7296.5,1.185,20324.5,21445.3,116.2445,4538.43,70.6,1.57
2021-12-06,333.873647,27.18,4431.89944,280.126,7296.5,1.185,20478.1,21589.8,116.1674,4591.67,70.6,1.57
2021-12-07,333.873647,21.89,4431.89944,280.126,7296.5,1.185,20478.1,21589.8,115.8815,4686.75,70.6,1.57
2021-12-08,333.873647,19.90,4431.89944,280.126,7296.5,1.185,20478.1,21589.8,115.4694,4701.21,70.6,1.57


In [15]:
output_file_path = product['data']
parent_file_path = Path(output_file_path).parent
series_info_file_path = str(parent_file_path) + "/fred_series_info.csv"
print(f"Writing to {output_file_path} \n {series_info_file_path}")

Writing to /home/vgaurav/market_watch/output/data/raw/fred_series_data.csv 
 /home/vgaurav/market_watch/output/data/raw/fred_series_info.csv


In [16]:
series_wide_df.to_csv(output_file_path)
series_info_df.to_csv(series_info_file_path)